# Introduction

이 리포지토리에 오신 것을 환영합니다. RAG(검색 증강 생성, 검색의 힘과 AI 생성을 결합하여 사용자 쿼리에 답하는 기술)의 작동 방식을 이해하는 일련의 노트북으로 안내해 드리겠습니다. 이 노트북의 마지막 부분에서는 Azure AI Search와 함께 작업하며, 이 조합으로 마법이 일어나는 이유를 이해하게 될 것입니다:

1) 단일 에이전트 
2) Azure OpenAI 모델 
3) 매우 상세한 프롬프트

하지만 기본부터 시작해야 하므로 Azure AI Search와 그 작동 방식부터 시작하겠습니다.

# Load and Enrich multiple file types Azure AI Search

이 Jupyter 노트북에서는 지정된 Azure 블롭에서 검색 가능한 콘텐츠를 잠금 해제하기 위한 강화 단계를 만들고 실행합니다. 이미지 및 애플리케이션 파일과 같은 Azure 저장소의 혼합 콘텐츠에 대한 작업을 수행하며, Azure 인지 검색에서 검색 가능한 텍스트 정보를 분석하고 추출하는 스킬셋을 사용합니다. 참조 샘플은 자습서: Python 및 AI를 사용하여 Azure 블롭에서 검색 가능한 콘텐츠 생성에서 찾을 수 있습니다 [Tutorial: Use Python and AI to generate searchable content from Azure blobs](https://docs.microsoft.com/azure/search/cognitive-search-tutorial-blob-python).

이 데모에서는 Azure AI 검색 샘플 데이터에서 ~9.8k Contoso Electronics HR PDF를 사용하겠습니다. 샘플파일은 .\data\hrdocs 폴더에 존재합니다. 여기서는 PDF 파일만 사용되었지만 훨씬 더 큰 규모에서 이 작업을 수행할 수 있으며 Azure AI Search는 다음과 같은 다양한 파일 형식을 지원합니다. 
- Microsoft Office(DOCX/DOC, XSLX/XLS, PPTX/PPT, MSG), HTML, XML, ZIP 및 일반 텍스트 파일(JSON 포함).

이 노트북은 검색 서비스에서 다음 개체를 만듭니다.

+ data source
+ search index
+ skillset
+ indexer

이 노트북에서는 Search REST APIs를 호출하지만, Python용 Azure SDK의 Azure.Search.Documents 클라이언트 라이브러리를 사용하여 동일한 단계를 수행할 수도 있습니다. 자세한 내용은 이 Python 빠른 시작을 참조하세요.

이 노트북을 실행하려면 README에서 Azure 서비스를 이미 만들었어야 합니다. 이 작업을 완료한 후에는 모든 셀을 실행할 수 있지만 인덱서가 완료되고 검색 인덱스가 로드될 때까지 쿼리에서 결과를 반환하지 않습니다.

We recommend running each step and making sure it completes before moving on.

![cog-search](./images/Cog-Search-Enrich.png)

In [ ]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# Name of the container in your Blob Storage Datasource ( in credentials.env)
BLOB_CONTAINER_NAME = "hrdocs"

In [ ]:
# Define the names for the data source, skillset, index and indexer
datasource_name = "cogsrch-datasource-hrdocs"
index_name = "cogsrch-index-hrdocs"
skillset_name = "cogsrch-skillset-hrdocs"
indexer_name = "cogsrch-indexer-hrdocs"

In [ ]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Create Data Source (Blob container with the Contoso Electronics HR PDFs)

In [ ]:
# The following code sends the json paylod to Azure Search engine to create the Datasource

datasource_payload = {
    "name": datasource_name,
    "description": "hr document files to demonstrate AI search capabilities.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "dataDeletionDetectionPolicy" : {
        "@odata.type" :"#Microsoft.Azure.Search.NativeBlobSoftDeleteDeletionDetectionPolicy" # this makes sure that if the item is deleted from the source, it will be deleted from the index
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

- 201 - Successfully created
- 204 - Succesfully overwritten
- 40X - Authentication Error

For information on Change and Delete file detection please see [HERE](https://learn.microsoft.com/en-us/azure/search/search-howto-index-changed-deleted-blobs?tabs=rest-api)

In [ ]:
# If you have a 403 code, probably you have a wrong endpoint or key, you can debug by uncomment this
# r.text

## Create Index

Azure AI Search에서 검색 인덱스는 검색 엔진에서 인덱싱, 전체 텍스트 검색, 벡터 검색, 하이브리드 검색 및 필터링된 쿼리를 위해 사용할 수 있는 검색 가능한 콘텐츠입니다. 인덱스는 스키마에 의해 정의되고 검색 서비스에 저장되며, 데이터 가져오기는 두 번째 단계로 이어집니다. 이 콘텐츠는 최신 검색 애플리케이션에서 기대되는 밀리초 단위의 응답 시간을 위해 필요한 기본 데이터 저장소와는 별도로 검색 서비스 내에 존재합니다. 인덱서 기반 인덱싱 시나리오를 제외하고 검색 서비스는 소스 데이터에 연결하거나 쿼리하지 않습니다.

Reference:

https://learn.microsoft.com/en-us/azure/search/search-what-is-an-index

아래에서 벡터 저장소를 만드는 방법을 확인하세요. Azure AI Search에서 벡터 저장소에는 벡터 및 비벡터 필드를 정의하는 인덱스 스키마, 임베딩 공간을 만드는 알고리즘에 대한 벡터 구성, 쿼리 요청에 사용되는 벡터 필드 정의에 대한 설정이 있습니다. 

또한 결과 세트에 대한 의미론적 순위를 설정하여 의미론적으로 가장 관련성이 높은 결과를 스택의 맨 위로 끌어올릴 수 있습니다. 또한 가장 관련성이 높은 용어와 구문, 의미론적 답변에 대한 하이라이트가 포함된 의미론적 캡션을 얻을 수도 있습니다.

In [ ]:
# Create an index
# Queries operate over the searchable fields and filterable fields in the index
index_payload = {
    "name": index_name,
    "vectorSearch": {
        "algorithms": [
            {
                "name": "myalgo",
                "kind": "hnsw"
            }
        ],
        "vectorizers": [
            {
                "name": "openai",
                "kind": "azureOpenAI",
                "azureOpenAIParameters":
                {
                    "resourceUri" : os.environ['AZURE_OPENAI_ENDPOINT'],
                    "apiKey" : os.environ['AZURE_OPENAI_API_KEY'],
                    "deploymentId" : os.environ['EMBEDDING_DEPLOYMENT_NAME']
                }
            }
        ],
        "profiles": [
            {
                "name": "myprofile",
                "algorithm": "myalgo",
                "vectorizer":"openai"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    },
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "analyzer": "keyword", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false","facetable": "false"},
        {"name": "ParentKey", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},   
        {"name": "chunk","type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {
            "name": "chunkVector",
            "type": "Collection(Edm.Single)",
            "dimensions": 1536, # IMPORTANT: Make sure these dimmensions match your embedding model name
            "vectorSearchProfile": "myprofile",
            "searchable": "true",
            "retrievable": "true",
            "filterable": "false",
            "sortable": "false",
            "facetable": "false"
        }
    ]
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)


In [ ]:
r.text

#### Semantic Search capabilities

위의 인덱스 페이로드에서 볼 수 있듯이 '시맨틱 구성'이 있습니다.

시맨틱 랭커는 텍스트 기반 쿼리에 대한 초기 BM25 랭킹 또는 RRF 랭킹 검색 결과의 품질을 개선하는 쿼리 관련 기능 모음입니다. 

검색 서비스에서 이 기능을 활성화하면 시맨틱 랭킹은 두 가지 방식으로 쿼리 실행 파이프라인을 확장합니다.

    첫째, BM25 또는 RRF를 사용하여 채점된 초기 결과 세트에 보조 순위를 추가합니다. 이 보조 순위는 의미론적으로 가장 관련성이 높은 결과를 표시하기 위해 Microsoft Bing에서 채택한 다국어 딥 러닝 모델을 사용합니다.
    
    둘째, 응답에서 캡션과 답변을 추출하여 반환하며, 이를 검색 페이지에 렌더링하여 사용자의 검색 환경을 개선할 수 있습니다.

더 자세한 설명과 제한 사항은 [여기](https://learn.microsoft.com/en-us/azure/search/semantic-ranking)를 참조하세요.
 

## Create Skillset - OCR, Text Splitter, AzureOpenAIEmbeddingSkill

We need to create now the skillset. This is a set of steps in which we use AI Services to enrich the documents by extracting information, applying OCR, splitting, and embedding chunks, among other skills.

https://learn.microsoft.com/en-us/azure/search/cognitive-search-working-with-skillsets

https://learn.microsoft.com/en-us/azure/search/cognitive-search-predefined-skills



아래에서 인덱스 투영을 사용하고 있음을 알 수 있습니다. 기본적으로 스킬셋 내에서 처리된 하나의 문서는 검색 인덱스의 단일 문서에 매핑됩니다. 즉, 입력 텍스트의 청킹을 수행한 다음 각 청크에 대해 보강을 수행하면 출력 필드 매핑을 통해 매핑된 인덱스의 결과는 생성된 보강의 배열이 됩니다. **인덱스 투영을 사용하면 각 강화 데이터 청크를 자체 검색 문서에 매핑할 컨텍스트를 정의할 수 있습니다**. 이를 통해 문서의 보강된 데이터를 검색 인덱스에 일대다 매핑할 수 있습니다.
    
매개변수: `"projectionMode": "skipIndexingParentDocuments"`를 사용하면 상위 문서의 인덱싱을 건너뛰고 청크와 그 벡터가 포함된 인덱스만 유지할 수 있습니다.

### Content Lifecycle
인덱서 데이터 소스가 변경 추적 및 삭제 감지를 지원하는 경우, 인덱싱 프로세스는 기본 색인(부모 문서)과 보조 색인(청크)을 동기화하여 이러한 변경 사항을 포착할 수 있습니다.


인덱서 및 스킬셋을 실행할 때마다 스킬셋 또는 기본 소스 데이터가 변경되면 인덱스 예상이 업데이트됩니다. 색인기가 포착한 모든 변경 사항은 보강 프로세스를 통해 인덱스의 투영으로 전파되어 투영 데이터가 원본 데이터 소스의 콘텐츠를 최신으로 표현하도록 보장합니다. 이렇게 하면 몇 주에 걸친 프로그래밍과 콘텐츠를 동기화하기 위한 많은 골칫거리를 줄일 수 있습니다.

In [ ]:
# Create a skillset
skillset_payload = {
    "name": skillset_name,
    "description": "e2e Skillset for RAG - Files",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
            "description": "Extract text (plain and structured) from image.",
            "context": "/document/normalized_images/*",
            "defaultLanguageCode": "en",
            "detectOrientation": True,
            "inputs": [
                {
                  "name": "image",
                  "source": "/document/normalized_images/*"
                }
            ],
                "outputs": [
                {
                  "name": "text",
                  "targetName" : "images_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
            "description": "Create merged_text, which includes all the textual representation of each image inserted at the right location in the content field. This is useful for PDF and other file formats that supported embedded images.",
            "context": "/document",
            "insertPreTag": " ",
            "insertPostTag": " ",
            "inputs": [
                {
                  "name":"text", "source": "/document/content"
                },
                {
                  "name": "itemsToInsert", "source": "/document/normalized_images/*/images_text"
                },
                {
                  "name":"offsets", "source": "/document/normalized_images/*/contentOffset"
                }
            ],
            "outputs": [
                {
                  "name": "mergedText", 
                  "targetName" : "merged_text"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",  # although it says "pages" it actally means chunks, not actual pages
            "maximumPageLength": 5000, # 5000 characters is default and a good choice
            "pageOverlapLength": 750,  # 15% overlap among chunks
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/merged_text"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "chunks"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.AzureOpenAIEmbeddingSkill",
            "description": "Azure OpenAI Embedding Skill",
            "context": "/document/chunks/*",
            "resourceUri": os.environ['AZURE_OPENAI_ENDPOINT'],
            "apiKey": os.environ['AZURE_OPENAI_API_KEY'],
            "deploymentId": os.environ['EMBEDDING_DEPLOYMENT_NAME'],
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/chunks/*"
                }
            ],
            "outputs": [
                {
                    "name": "embedding",
                    "targetName": "vector"
                }
            ]
        }
    ],
    "indexProjections": {
        "selectors": [
            {
                "targetIndexName": index_name,
                "parentKeyFieldName": "ParentKey",
                "sourceContext": "/document/chunks/*",
                "mappings": [
                    {
                        "name": "title",
                        "source": "/document/title"
                    },
                    {
                        "name": "name",
                        "source": "/document/name"
                    },
                    {
                        "name": "location",
                        "source": "/document/location"
                    },
                    {
                        "name": "chunk",
                        "source": "/document/chunks/*"
                    },
                    {
                        "name": "chunkVector",
                        "source": "/document/chunks/*/vector"
                    }
                ]
            }
        ],
        "parameters": {
            "projectionMode": "skipIndexingParentDocuments"
        }
    },
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

In [ ]:
print(r.text)

## Create and Run the Indexer - (runs the pipeline)

지금까지 만든 세 가지 구성 요소(데이터 원본, 스킬셋, 인덱스)는 인덱서에 대한 입력입니다. Azure Cognitive Search에서 인덱서를 만드는 것이 전체 파이프라인을 작동시키는 이벤트입니다.

In [ ]:
# Create an indexer
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "disabled": None,
    "schedule" : None, # How often do you want to check for new content in the data source
    "fieldMappings": [
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "title"
        },
        {
          "sourceFieldName" : "metadata_storage_name",
          "targetFieldName" : "name"
        },        
        {
          "sourceFieldName" : "metadata_storage_path",
          "targetFieldName" : "location"
        }        
    ],
    "outputFieldMappings":[],
    "parameters":
    {
        "maxFailedItems": -1,
        "maxFailedItemsPerBatch": -1,
        "configuration":
        {
            "dataToExtract": "contentAndMetadata",
            "imageAction": "generateNormalizedImages"
        }
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

In [ ]:
# Uncomment if you find an error
r.text

Note : 400 권한 없음 오류가 발생하는 경우, 쿼리 키가 아닌 Azure Search 관리 키를 사용하고 있는지 확인하세요.

In [ ]:
# Optionally, get indexer status to confirm that it's running
try:
    r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                     "/status", headers=headers, params=params)
    # pprint(json.dumps(r.json(), indent=1))
    print(r.status_code)
    print("Status:",r.json().get('lastResult').get('status'))
    print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
    print(r.ok)
    
except Exception as e:
    print("Wait a few seconds until the process starts and run this cell again.")

**실행된 인덱스를 Azure Portal에서 확인합니다.**

# References

- https://learn.microsoft.com/en-us/azure/search/cognitive-search-tutorial-blob
- https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search
- https://learn.microsoft.com/en-us/azure/search/search-get-started-vector

# NEXT
이제 인덱스가 로드되었으므로 다음 노트북 3에서는 인덱스 쿼리를 수행하고 Azure AI Search의 재랭커 시맨틱 점수를 기반으로 결과를 정렬한 다음 OpenAI를 사용하여 결과를 이해하고 가능한 최상의 답변을 제공하겠습니다.